In [1]:
import random
from typing import Literal, cast

import torch as t
import torch.nn.functional as F
from torchvision.datasets import ImageNet
from torchvision.models.resnet import ResNet
from tqdm import tqdm
from plotly import express as px, graph_objects as go

from src import load_latest
from src.utils import seed

In [2]:
model: ResNet = t.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=True)
model.eval();

Using cache found in /home/matthewbaggins/.cache/torch/hub/pytorch_vision_v0.10.0
/home/matthewbaggins/code/deep_learning_curriculum/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/matthewbaggins/code/deep_learning_curriculum/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Download an example image from the pytorch website
from urllib import request
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
request.urlretrieve(url, filename)

('dog.jpg', <http.client.HTTPMessage at 0x7fb8b8947c90>)

In [4]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
assert isinstance(input_tensor, t.Tensor)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if t.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with t.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = t.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([ 1.6339e-02, -1.5506e+00,  3.2032e-01, -2.0590e+00, -8.5789e-01,
         1.7853e+00,  1.4694e+00,  2.1625e+00,  4.4890e+00,  8.2922e-01,
        -5.7825e+00, -3.4975e+00, -4.0629e+00, -4.7529e+00, -3.8074e+00,
        -4.7250e+00, -1.2607e+00,  2.9643e-01, -2.0471e+00, -5.3099e-01,
        -3.5982e+00, -8.1666e-01, -2.7665e+00, -1.2774e+00, -3.4204e+00,
        -1.9035e+00, -3.0009e+00, -1.3474e+00, -1.8383e+00,  1.3947e+00,
        -2.0109e+00, -1.4142e+00, -2.3272e+00, -1.8205e+00, -1.1873e-01,
        -3.4104e+00, -1.6544e+00, -3.4491e+00, -2.6464e+00, -2.7418e+00,
        -2.2195e+00, -3.6512e+00, -4.1251e+00, -5.5947e+00, -1.7520e+00,
        -1.6909e+00, -9.8139e-01, -2.1243e+00, -3.5150e+00, -1.3327e+00,
        -1.1343e+00, -1.1564e+00, -2.2390e-02, -8.5852e-01, -1.2917e+00,
        -2.8688e+00,  6.5904e-01, -1.7179e+00, -1.2448e+00, -2.3358e+00,
        -5.8504e-02, -1.9220e+00, -2.5971e+00, -1.8031e+00, -1.5120e+00,
        -1.0845e+00, -4.0847e-01, -1.3099e+00, -9.4

In [5]:
# input_image.show()

In [6]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-12-06 07:27:38--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.4’

imagenet_classes.tx 100%[===================>]  10,23K  --.-KB/s    in 0,004s  

2023-12-06 07:27:38 (2,46 MB/s) - ‘imagenet_classes.txt.4’ saved [10472/10472]



In [7]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = t.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8845872282981873
Arctic fox 0.04575274884700775
white wolf 0.04435020312666893
Pomeranian 0.00561820063740015
Great Pyrenees 0.004655089695006609


## Features visualization

In [8]:
for _, param in model.named_parameters():
    param.requires_grad_(False)
assert all(not param.requires_grad for _, param in model.named_parameters())

In [9]:
model.to(device=t.device("cuda"));

Generate artificial image that maximizes a neuron in `layer1`.

In [10]:
seed(42)
lr = 1e-3
N = 1_0_000
act_inds = (0, 0, 22, 20,)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x)))))
    act = acts[act_inds]
    act.backward()
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

100%|██████████| 10000/10000 [00:10<00:00, 943.60it/s]


In [14]:
seed(42)
lr = 1e2
N = 1_0_000
act_inds = (0, 0, 0, 0,)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x)))))
    acts.backward()
    
    for act in acts.flatten():
        
    
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

  0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: grad can be implicitly created only for scalar outputs

In [13]:
Image.fromarray(x[0,0].detach().cpu().numpy()).show()

`layer2`

In [119]:
seed(42)
lr = 1e2
N = 1_0000
act_inds = (0, 0, 22, 20,)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer2(model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x))))))
    act = acts[act_inds]
    act.backward()
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

100%|██████████| 10000/10000 [00:16<00:00, 618.17it/s]


In [120]:
Image.fromarray(x[0,0].detach().cpu().numpy()).show()

`layer3`

In [123]:
seed(42)
lr = 1e2
N = 1_0000
act_inds = (0, 0, 10, 10,)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer3(model.layer2(model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x)))))))
    act = acts[act_inds]
    act.backward()
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

100%|██████████| 10000/10000 [00:27<00:00, 357.67it/s]


In [124]:
Image.fromarray(x[0,0].detach().cpu().numpy()).show()

`layer4`

In [126]:
seed(42)
lr = 1e2
N = 1_0000
act_inds = (0, 0, 5, 5,)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer4(model.layer3(model.layer2(model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x))))))))
    act = acts[act_inds]
    act.backward()
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

100%|██████████| 10000/10000 [00:37<00:00, 270.17it/s]


In [127]:
Image.fromarray(x[0,0].detach().cpu().numpy()).show()

`fc`

In [135]:
seed(42)
lr = 1e2
N = 1_0000
act_inds = (0, 0)
x = t.randn_like(input_batch)

for i in tqdm(range(N)):
    x.requires_grad_(True)
    acts = model.layer4(model.layer3(model.layer2(model.layer1(model.maxpool(model.relu(model.bn1(model.conv1(x))))))))
    acts = model.fc(model.avgpool(acts).flatten(1))
    act = acts[act_inds]
    act.backward()
    with t.no_grad():
        x = x + lr * cast(t.Tensor, x.grad)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [131]:
Image.fromarray(x[0,0].detach().cpu().numpy()).show()